## Entities

We identified four basic entities in our schema:

1. <span style="color: var(--vscode-foreground);">Location</span>

2\. Hospital

3\. Hospital Quality

4\. Weekly Report

We chose these entities to effectively separate information like hospital details and location from weekly reports and quality ratings. This approach reduces redundancy by making sure that each piece of information is stored only once.

## 1\. Location

Table to store unique location information:

 The 'Location' entity stores unique geographical information to avoid redundancy when multiple hospitals <span style="color: var(--vscode-foreground);">share the same location details, which helps to minimize duplication.</span>

In [12]:
CREATE TABLE location (
    id SERIAL PRIMARY KEY,
    city TEXT,
    state TEXT,
    zip_code TEXT,
    address TEXT,
    latitude FLOAT,
    longitude FLOAT,
    fips_code TEXT,
    CONSTRAINT location_unique UNIQUE (city, state, zip_code, address, latitude, longitude)  -- Add unique constraint
);

CREATE TABLE

Total execution time: 00:00:00.044

## 2\. Hospital

Table for hospital information:

<span style="color: var(--vscode-foreground);">This table represents the 'Hospital' entity and stores information about each hospital.&nbsp;</span> <span style="color: var(--vscode-foreground);">We use 'hospital_pk' as the primary key to uniquely identify each hospital.&nbsp;</span> <span style="color: var(--vscode-foreground);">The 'location_id' is a foreign key referencing the 'location' table, linking each hospital to its location.</span>

In [13]:
CREATE TABLE hospital(
    hospital_pk TEXT PRIMARY KEY,
    hospital_name TEXT NOT NULL,
    location_id INTEGER REFERENCES location(id) ON DELETE SET NULL
);

CREATE TABLE

Total execution time: 00:00:00.030

## 3\. Hospital Quality

Table to store the quality ratings of each hospital:

<span style="color: var(--vscode-foreground);">This table keeps track of quality ratings for each hospital over time, where</span> <span style="color: var(--vscode-foreground);">'facility_id' links back to the 'hospital' table.&nbsp;</span>   <span style="color: var(--vscode-foreground);">We can see how a hospital's rating changes by date through this table.</span>

In [14]:
CREATE TABLE hospital_quality (
    id SERIAL PRIMARY KEY,
    facility_id TEXT REFERENCES hospital(hospital_pk) ON DELETE CASCADE, 
    quality_rating INTEGER CHECK (quality_rating BETWEEN 1 AND 5),  -- Matches "Hospital overall rating" (1-5 scale)
    rating_date DATE NOT NULL,  
    ownership TEXT,           
    hospital_type TEXT,         
    provides_emergency_services BOOLEAN  
);

CREATE TABLE

Total execution time: 00:00:00.034

## 4\. Weekly Report

Table to store weekly data reports for each hospital:

This table stores weekly report dates for each hospital, where 'hospital\_weekly\_id' links to the 'hospital' table. We ensure there's only one report per hospital each week.

In [15]:
CREATE TABLE weekly_report (
    id SERIAL PRIMARY KEY,
    hospital_weekly_id TEXT REFERENCES hospital(hospital_pk) ON DELETE CASCADE,  
    collection_week DATE NOT NULL,
    all_adult_hospital_beds_7_day_avg FLOAT,  
    all_pediatric_inpatient_beds_7_day_avg FLOAT,
    total_icu_beds_7_day_avg FLOAT,
    all_adult_hospital_inpatient_bed_occupied_7_day_avg FLOAT,  
    all_pediatric_inpatient_bed_occupied_7_day_avg FLOAT,  
    icu_beds_used_7_day_avg FLOAT, 
    inpatient_beds_used_covid_7_day_avg FLOAT,  
    staffed_icu_adult_patients_confirmed_covid_7_day_avg FLOAT,  
    UNIQUE(hospital_weekly_id, collection_week)  
);

CREATE TABLE

Total execution time: 00:00:00.047

## Summarizing relationships:

| Table | Primary Key | Foreign Key | Relationship |
| --- | --- | --- | --- |
| `location` | `id` | None | Standalone table for unique locations |
| `hospital` | `hospital_pk` | `location_id` references `location(id)` | Links each hospital to a unique location |
| `hospital_quality` | `id` | `facility_id` references `hospital(hospital_pk)` | Links each quality rating to a hospital |
| `weekly_report` | `id` | `hospital_weekly_id` references `hospital(hospital_pk)` | Links each weekly report to a hospital |
| <br> |  |  | <br> |

We designed our schema this way to separate data into different tables to keep things organized and avoid duplicates:

\- The 'hospital' table stores each hospital's info once.

\- The 'location' table holds location details that can be shared by multiple hospitals.

\- The 'hospital\_quality' table tracks quality ratings over time without repeating hospital info.

\- The 'weekly\_report' tables store time-based patient data linked to hospitals.

By linking tables with foreign keys, we connect related data without storing the same information multiple times. This makes our database efficient and easier to maintain.